## Test OpenPCDet transformation

In [ ]:
import numpy as np
import copy
import pickle
from pathlib import Path

def get_calib_from_pkl(dataset_path, lidar_idx):
    """
    Load calibration data from pickle file for specific frame
    
    Args:
        dataset_path: Path to kitti_val_dataset.pkl
        lidar_idx: Frame index to load calibration for
        
    Returns:
        dict: Calibration data in OpenPCDet format
    """
    with open(dataset_path, 'rb') as f:
        dataset = pickle.load(f)
    
    # Find frame with matching lidar_idx
    calib_data = None
    for frame in dataset:
        if frame['point_cloud']['lidar_idx'] == lidar_idx:
            calib_data = frame['calib']
            break
    
    if calib_data is None:
        raise ValueError(f"No calibration data found for frame {lidar_idx}")
        
    # Convert to OpenPCDet format
    return {
        'P2': calib_data['P2'][:3],  # 3 x 4
        'R0': calib_data['R0_rect'][:3, :3],  # 3 x 3
        'Tr_velo2cam': calib_data['Tr_velo_to_cam'][:3],  # 3 x 4
    }

class Calibration(object):
    def __init__(self, calib_file):
        if not isinstance(calib_file, dict):
            calib = get_calib_from_pkl(calib_file)
        else:
            calib = calib_file

        self.P2 = calib['P2']  # 3 x 4
        self.R0 = calib['R0']  # 3 x 3
        self.V2C = calib['Tr_velo2cam']  # 3 x 4

    def cart_to_hom(self, pts):
        """
        :param pts: (N, 3 or 2)
        :return pts_hom: (N, 4 or 3)
        """
        pts_hom = np.hstack((pts, np.ones((pts.shape[0], 1), dtype=np.float32)))
        return pts_hom

    def lidar_to_rect(self, pts_lidar):
            """
            :param pts_lidar: (N, 3)
            :return pts_rect: (N, 3)
            """
            pts_lidar_hom = self.cart_to_hom(pts_lidar)
            pts_rect = np.dot(pts_lidar_hom, np.dot(self.V2C.T, self.R0.T))
            return pts_rect

def boxes3d_lidar_to_kitti_camera(boxes3d_lidar, calib):
    """
    :param boxes3d_lidar: (N, 7) [x, y, z, dx, dy, dz, heading], (x, y, z) is the box center
    :param calib:
    :return:
        boxes3d_camera: (N, 7) [x, y, z, h, w, l, r] in rect camera coords
    """
    boxes3d_lidar_copy = copy.deepcopy(boxes3d_lidar)
    xyz_lidar = boxes3d_lidar_copy[:, 0:3]
    h, w, l = boxes3d_lidar_copy[:, 3:4], boxes3d_lidar_copy[:, 4:5], boxes3d_lidar_copy[:, 5:6]
    r = boxes3d_lidar_copy[:, 6:7]

    xyz_lidar[:, 2] -= h.reshape(-1) / 2
    xyz_cam = calib.lidar_to_rect(xyz_lidar)
    # xyz_cam[:, 1] += h.reshape(-1) / 2
    r = -r - np.pi / 2
    return np.concatenate([xyz_cam, h, w, l, r], axis=-1)

if __name__ == "__main__":

    dataset_path = Path("validation_pickle/kitti_val_dataset.pkl")
    lidar_idx = "000033"
    calib_data = get_calib_from_pkl(dataset_path, lidar_idx)

    calib = Calibration(calib_data)
    
    # 1.48 1.6400006786508703 3.5899532324076224 10.438960000000002 7.107391999999999 -0.9011168479919434 -3.140806679384028
    test_box = np.array([[10.439, 7.107, -0.901, 1.48, 1.64, 3.59, -3.141]])

    box_cam = boxes3d_lidar_to_kitti_camera(test_box, calib)

    print("Input (LiDAR):", test_box[0])
    print("Output (Camera):", box_cam[0])